In [ ]:
from tensorflow import random
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import os
import generate_data.generate_data

img_size = 80
batch_size = 64
epochs = 100

train_data_generated, valid_data_generated, test_data_generated = generate_data(img_size, batch_size)
num_classes = len(os.listdir(os.path.abspath("consolidated")))

keras.backend.clear_session()
random.set_seed(42)

inputs = keras.Input(shape=(img_size, img_size, 3))

x = layers.Conv2D(16, (5, 5), padding="same", activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(128, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation="relu")(x)

output = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, output, name="bird_classifier")

model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(lr=1e-6),
    metrics=["accuracy"]
)

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 20)
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data_generated,
          validation_data=valid_data_generated,
          epochs=100,
          verbose=2,
          callbacks=[lr_schedule]
)

In [ ]:
plt.semilogx(history.history['lr'], history.history['loss'])
plt.axis([1e-6, 0.05, 0, 7])

1e-3

## Final Train

In [ ]:
keras.backend.clear_session()
random.set_seed(42)
num_classes = len(os.listdir(os.path.abspath("consolidated")))

inputs = keras.Input(shape=(img_size, img_size, 3))

x = layers.Conv2D(16, (5, 5), padding="same", activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(128, (5, 5), padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation="relu")(x)

output = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, output, name="bird_classifier")

early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    "mymodel.h5", monitor='val_loss', verbose=0, save_best_only=True
)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data_generated,
          validation_data=valid_data_generated,
          epochs=150,
          verbose=2,
          callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
keras.models.save_model(model, "mymodel.h5", save_format="h5")